In [1]:
import pandas as pd
import json
from glob import glob

## Dataset

[Yelp Dataset](https://www.yelp.com/dataset)
* yelp_academic_dataset_business.json
* yelp_academic_dataset_review.json
* yelp_academic_dataset_user.json
* yelp_academic_dataset_checkin.json
* yelp_academic_dataset_tip.json

In [2]:
pd.options.display.max_colwidth = None

In [3]:
resource_path = "Dataset/"
result_path = "Dataset/Done/"

## Restaurant Info
Contains business data including location data, attributes, and categories.

#### 所有店家資料

In [5]:
all_shops = pd.read_json(resource_path + "yelp_academic_dataset_business.json", lines=True )

print(all_shops.keys(),end="\n-------------------------------------------------------------\n")
print("資料筆數:" + str(len(all_shops)),end="\n-------------------------------------------------------------\n")
all_shops[['attributes']].head(20)

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')
-------------------------------------------------------------
資料筆數:160585
-------------------------------------------------------------


,attributes
0,"{'RestaurantsTableService': 'True', 'WiFi': 'u'free'', 'BikeParking': 'True', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsReservations': 'False', 'WheelchairAccessible': 'True', 'Caters': 'True', 'OutdoorSeating': 'True', 'RestaurantsGoodForGroups': 'True', 'HappyHour': 'True', 'BusinessAcceptsBitcoin': 'False', 'RestaurantsPriceRange2': '2', 'Ambience': '{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}', 'HasTV': 'True', 'Alcohol': ''beer_and_wine'', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}', 'DogsAllowed': 'False', 'RestaurantsTakeOut': 'True', 'NoiseLevel': 'u'average'', 'RestaurantsAttire': ''casual'', 'RestaurantsDelivery': 'None'}"
1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAttire': 'u'casual'', 'GoodForKids': 'True', 'BikeParking': 'False', 'OutdoorSeating': 'False', 'Ambience': '{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}', 'Caters': 'True', 'RestaurantsReservations': 'False', 'RestaurantsDelivery': 'False', 'HasTV': 'False', 'RestaurantsGoodForGroups': 'False', 'BusinessAcceptsCreditCards': 'True', 'NoiseLevel': 'u'average'', 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '2', 'WiFi': 'u'free'', 'BusinessParking': '{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'Alcohol': 'u'beer_and_wine'', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': True}'}"
2,"{'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'ByAppointmentOnly': 'False', 'BikeParking': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}'}"
3,"{'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'True', 'GoodForKids': 'True', 'ByAppointmentOnly': 'False'}"
4,"{'GoodForKids': 'False', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'BusinessAcceptsCreditCards': 'True'}"
5,"{'GoodForKids': 'True', 'Alcohol': 'u'none'', 'RestaurantsGoodForGroups': 'True', 'RestaurantsReservations': 'True', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'RestaurantsAttire': 'u'casual'', 'BikeParking': 'True', 'RestaurantsPriceRange2': '2', 'HasTV': 'False', 'NoiseLevel': 'u'average'', 'WiFi': 'u'no'', 'RestaurantsTakeOut': 'True', 'Caters': 'False', 'OutdoorSeating': 'False', 'Ambience': '{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}', 'DogsAllowed': 'False', 'RestaurantsDelivery': 'True'}"
6,"{'AcceptsInsurance': 'True', 'BusinessAcceptsCreditCards': 'True', 'ByAppointmentOnly': 'True'}"
7,None
8,"{'RestaurantsPriceRange2': '1', 'ByAppointmentOnly': 'False', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'BusinessAcceptsCreditCards': 'True', 'DogsAllowed': 'True', 'RestaurantsDelivery': 'None', 'BusinessAcceptsBitcoin': 'False', 'BikeParking': 'True', 'RestaurantsTakeOut': 'None', 'WheelchairAccessible': 'True'}"
9,"{'BusinessAcceptsCreditCards': 'False', 'RestaurantsPriceRange2': '3', 'ByAppointmentOnly': 'False', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}'}"


In [ ]:
all_shops = pd.read_json(resource_path + "yelp_academic_dataset_business.json", lines=True )

print(all_shops.keys(),end="\n-------------------------------------------------------------\n")
print("資料筆數:" + str(len(all_shops)),end="\n-------------------------------------------------------------\n")
all_shops[['business_id', 'name', 'address', 'city', 'state','stars', 'review_count']].head()

#### 餐廳資料

In [ ]:
df_list = []

for i in range(len(all_shops)):
    if "Restaurants" in str(all_shops.loc[i, 'categories']):
        df_list.append(all_shops.loc[[i]])
        
restaurants = pd.concat(df_list).reset_index(drop=True)

In [ ]:
print("資料筆數:" + str(len(restaurants)))
print("資料是否有重複:" + str(restaurants['business_id'].duplicated().any()))
print("資料是否有空值:" + str(restaurants.isnull().values.any()))

In [ ]:
# 把所有餐廳資料存成csv檔
restaurants.to_csv(result_path + "restaurants.csv", index = False)

In [ ]:
business_id = restaurants['business_id']

## Reviews
Contains full review text data including the user_id that wrote the review and the business_id the review is written for.

#### 所有店家評論

In [ ]:
rv_chunk_data = pd.read_json(resource_path + "yelp_academic_dataset_review.json", lines=True, chunksize=100000 )
rv_chunk_list = []

for chunk in rv_chunk_data:
    rv_chunk_list.append(chunk)
    
len(rv_chunk_list)

In [ ]:
print(rv_chunk_list[0].keys(),end="\n-------------------------------------------------------------\n")
print("評論筆數:" + str((len(rv_chunk_list)-1)*100000 + len(rv_chunk_list[-1])),end="\n-------------------------------------------------------------\n")
rv_chunk_list[0].loc[0:10, ['review_id', 'user_id', 'business_id', 'stars', 'text', 'date']]

#### 餐廳評論

In [ ]:
# 把所有餐廳評論存成多個csv檔
counts = 0
for i in range(0, len(rv_chunk_list)):
    reviews = pd.merge(business_id, rv_chunk_list[i], on="business_id", how="inner")
    reviews.to_csv(result_path + f"reviews/reviews{i+1}.csv", index = False)
    counts += len(reviews)
    
print("評論筆數:" + str(counts))

In [ ]:
# 把多個csv檔存成一個csv檔
files = glob(result_path + "reviews/reviews*.csv")
print(len(files))
df = pd.concat((pd.read_csv(file) for file in files)).reset_index(drop=True)
df.to_csv(result_path + "reviews.csv", index = False)

## User Info
User data including the user's friend mapping and all the metadata associated with the user.

In [ ]:
ci_chunk_data = pd.read_json(resource_path + "yelp_academic_dataset_user.json", lines=True, chunksize=100000)
ci_chunk_list = []

for chunk in ci_chunk_data:
    ci_chunk_list.append(chunk)

In [ ]:
print(ci_chunk_list[0].keys(),end="\n-------------------------------------------------------------\n")
print("資料筆數:" + str((len(ci_chunk_list)-1)*100000 + len(ci_chunk_list[-1])),end="\n-------------------------------------------------------------\n")
ci_chunk_list[0].loc[0:10, ['user_id', 'name', 'review_count', 'yelping_since', 'average_stars']]

In [ ]:
# 把所有使用者資訊存成多個csv檔
for i in range(0, len(ci_chunk_list)):
    ci_chunk_list[i].to_csv(result_path + f"user/user{i+1}.csv", index = False)

In [ ]:
# 把多個csv檔存成一個csv檔
files = glob(result_path + "user/user*.csv")
print(len(files))
df = pd.concat((pd.read_csv(file) for file in files)).reset_index(drop=True)
df.to_csv(result_path + "user.csv", index = False)

## Check in Info
Checkins on a business

In [ ]:
check = pd.read_json(resource_path + "yelp_academic_dataset_checkin.json",lines=True )
print(check.keys(),end="\n-------------------------------------------------------------\n")
print("資料筆數:" + str(len(check)),end="\n-------------------------------------------------------------\n")
check.head(2)

## Tips
Tips written by a user on a business. Tips are shorter than reviews and tend to convey quick suggestions.

#### 所有店家Tips

In [ ]:
all_tips = pd.read_json(resource_path + "yelp_academic_dataset_tip.json",lines=True )
print(all_tips.keys(),end="\n-------------------------------------------------------------\n")
print("資料筆數:" + str(len(all_tips)),end="\n-------------------------------------------------------------\n")
all_tips.head(5)

#### 餐廳Tips

In [ ]:
restaurants_tips = pd.merge(business_id, all_tips, on="business_id", how="inner")
print("資料筆數:" + str(len(restaurants_tips)),end="\n-------------------------------------------------------------\n")
restaurants_tips[['user_id', 'business_id', 'text', 'date']].head(10)